In [1]:
import random
from string import ascii_uppercase, ascii_letters
from pprint import pprint
from copy import deepcopy

In [2]:
## from http://users.telenet.be/d.rijmenants/en/enigmatech.htm
entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'  #(rotor right side)   
I     = 'EKMFLGDQVZNTOWYHXUSPAIBRCJ'
II    = 'AJDKSIRUXBLHWTMCQGZNPYFVOE'
III   = 'BDFHJLCPRTXVZNYEIWGAKMUSQO'
IV    = 'ESOVPZJAYQUIRHXLNFTGKDCMWB'
V     = 'VZBRGITYUPSDNHLXAWMJQOFECK'
## Additional rotors used by Kriegsmarine M3 and M4 only:
VI    = 'JPGVOUMFYQBENHZRDKASXLICTW'
VII   = 'NZJHGRCXMYSWBOUFAIVLPEKQDT'
VIII  = 'FKQHTLXOCBJSPDZRAMEWNIUYGV'

## reflectors, standard army & airforce
contacts   = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ' 
breflector = 'YRUHQSLDPXNGOKMIEBFZCWVJAT'
creflector = 'FVPJIAOYEDRZXWGCTKUQSBNMHL'

Notches
Rotor    Notch 	Window 	next left rotor steps when rotor steps from/to

I        Y      Q       Q -> R
II       M      E       E -> F
III      D      V       V -> W
IV       R      J       J -> K
V        H      Z       Z -> A
VI-VIII  H+U    Z, M    Z -> A and M -> N

Rotor 	Notch 	Window 	next left rotor steps when rotor steps from/to

I
II
III
IV
V
VI VII VIII 

	

Y
M
D
R
H
H + U

	

Q
E
V
J
Z
Z and M

	

Q -> R
E -> F
V -> W
J -> K
Z -> A
Z -> A and M -> N

In [3]:
raw_rotors = {'I':I, 'II':II, 'III':III, 'IV':IV, 'V':V}
print(raw_rotors)

{'I': 'EKMFLGDQVZNTOWYHXUSPAIBRCJ', 'II': 'AJDKSIRUXBLHWTMCQGZNPYFVOE', 'III': 'BDFHJLCPRTXVZNYEIWGAKMUSQO', 'IV': 'ESOVPZJAYQUIRHXLNFTGKDCMWB', 'V': 'VZBRGITYUPSDNHLXAWMJQOFECK'}


In [4]:
base_rotors = {'I':[], 'II':[], 'III':[], 'IV':[], 'V':[]}
for k in base_rotors.keys():
    base_rotors[k] = [ascii_uppercase.index(c) for c in raw_rotors[k]]
# rI = [ascii_uppercase.index(c) for c in I]
for k,v in base_rotors.items():
    print(k,':',v)

I : [4, 10, 12, 5, 11, 6, 3, 16, 21, 25, 13, 19, 14, 22, 24, 7, 23, 20, 18, 15, 0, 8, 1, 17, 2, 9]
II : [0, 9, 3, 10, 18, 8, 17, 20, 23, 1, 11, 7, 22, 19, 12, 2, 16, 6, 25, 13, 15, 24, 5, 21, 14, 4]
III : [1, 3, 5, 7, 9, 11, 2, 15, 17, 19, 23, 21, 25, 13, 24, 4, 8, 22, 6, 0, 10, 12, 20, 18, 16, 14]
IV : [4, 18, 14, 21, 15, 25, 9, 0, 24, 16, 20, 8, 17, 7, 23, 11, 13, 5, 19, 6, 10, 3, 2, 12, 22, 1]
V : [21, 25, 1, 17, 6, 8, 19, 24, 20, 15, 18, 3, 13, 7, 11, 23, 0, 22, 12, 9, 16, 14, 5, 4, 2, 10]


In [5]:
rev_rotors = {}

for r in raw_rotors.keys():
    working = {k:entry.index(v) for k,v in zip(raw_rotors[r],entry)}
    rev_rotors[r] = [working[k] for k in sorted(working.keys())]

for k,v in rev_rotors.items():
    print(k,':',v)

I : [20, 22, 24, 6, 0, 3, 5, 15, 21, 25, 1, 4, 2, 10, 12, 19, 7, 23, 18, 11, 17, 8, 13, 16, 14, 9]
II : [0, 9, 15, 2, 25, 22, 17, 11, 5, 1, 3, 10, 14, 19, 24, 20, 16, 6, 4, 13, 7, 23, 12, 8, 21, 18]
III : [19, 0, 6, 1, 15, 2, 18, 3, 16, 4, 20, 5, 21, 13, 25, 7, 24, 8, 23, 9, 22, 11, 17, 10, 14, 12]
IV : [7, 25, 22, 21, 0, 17, 19, 13, 11, 6, 20, 15, 23, 16, 2, 4, 9, 12, 1, 18, 10, 3, 24, 14, 8, 5]
V : [16, 2, 24, 11, 23, 22, 4, 13, 5, 19, 25, 14, 18, 12, 21, 9, 20, 3, 10, 6, 8, 0, 17, 15, 7, 1]


In [6]:
reflectors = {'Brf': {k:v for k,v in zip(entry, breflector)}, 'Crf': {k:v for k,v in zip(entry, creflector)}}
print(reflectors)

{'Brf': {'A': 'Y', 'B': 'R', 'C': 'U', 'D': 'H', 'E': 'Q', 'F': 'S', 'G': 'L', 'H': 'D', 'I': 'P', 'J': 'X', 'K': 'N', 'L': 'G', 'M': 'O', 'N': 'K', 'O': 'M', 'P': 'I', 'Q': 'E', 'R': 'B', 'S': 'F', 'T': 'Z', 'U': 'C', 'V': 'W', 'W': 'V', 'X': 'J', 'Y': 'A', 'Z': 'T'}, 'Crf': {'A': 'F', 'B': 'V', 'C': 'P', 'D': 'J', 'E': 'I', 'F': 'A', 'G': 'O', 'H': 'Y', 'I': 'E', 'J': 'D', 'K': 'R', 'L': 'Z', 'M': 'X', 'N': 'W', 'O': 'G', 'P': 'C', 'Q': 'T', 'R': 'K', 'S': 'U', 'T': 'Q', 'U': 'S', 'V': 'B', 'W': 'N', 'X': 'M', 'Y': 'H', 'Z': 'L'}}


In [7]:
def once_thru_scramble(ch, direction='forward', first_rotor='I', pos1=0, second_rotor='II', pos2=0, 
                       third_rotor='III', pos3=0):
    """ ch must be single ASCII character A-Z
    direction is either 'forward' or 'back' """
    if direction == 'forward':
        usedict = {k:v for k,v in base_rotors.items()}
    elif direction == 'back':
        usedict = {k:v for k,v in rev_rotors.items()}
    else:
        print('only forward or back for direction')
        return 'wtf'
    
    entry = ascii_uppercase
    ch = ch.upper()
    entry_pos = entry.index(ch)
    fst_pos_mod = (26 + pos1 - 0)%26
#     print(fst_pos_mod)
    fst_in = (entry_pos + fst_pos_mod)%26
#     print(fst_in)
    fst_out = usedict[first_rotor][fst_in]
#     print(fst_out)
    ch1o = entry[fst_out]
#     print('first rotor:  ', ch, '-->', ch1o)
    
    scd_pos_mod = (26 + pos2 - pos1)%26
    scd_in = (fst_out + scd_pos_mod)%26
    ch2i = entry[scd_in]
#     print('r1 to r2:     ', ch1o, '-->', ch2i)
#     print(scd_pos_mod, scd_in)
    scd_out = usedict[second_rotor][scd_in]
#     print(scd_out)
    ch2o = entry[scd_out]
#     print('second rotor: ', ch2i, '-->', ch2o)
    
    thd_pos_mod = (26 + pos3 - pos2)%26
    thd_in = (scd_out + thd_pos_mod)%26
    ch3i = entry[thd_in]
#     print('r2 to r3:     ', ch2o, '-->', ch3i)
    thd_out = usedict[third_rotor][thd_in]
#     print(thd_out)
    ch3o = entry[thd_out]
#     print('third rotor:  ', ch3i, '-->', ch3o)
    
    
    if direction == 'forward':
        print(f"{ch} -> (RR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (LR in) {ch3i} -> (LR out) {ch3o}")
    elif direction == 'back':
        print(f"{ch} -> (LR out) {ch1o} -> (MR in) {ch2i} -> (MR out) {ch2o} -> (RR in) {ch3i} -> (RR out) {ch3o}")
    
    
    return ch3o
    
    
once_thru_scramble('q')

Q -> (RR out) X -> (MR in) X -> (MR out) V -> (LR in) V -> (LR out) M


'M'

In [8]:
def full_scramble(in_ch, left_rotor='III', lpos=0, middle_rotor='II', mpos=0, right_rotor='I', rpos=0, rflector='Brf'):
    entry = ascii_uppercase
    in_ch = in_ch.upper()
    
    ## first run right to left through scrambler
    forward_run = once_thru_scramble(in_ch, 'forward', first_rotor=right_rotor, pos1=rpos, second_rotor=middle_rotor,
                                     pos2=mpos, third_rotor=left_rotor, pos3=lpos)
    
    ## reflector back around for return
    rfi_pos_mod = (26 + 0 - lpos)%26    ## the '0' is there to matching formatting of other position modifiers - reflector is not moved so it will always be 0
    rf_in = (entry.index(forward_run) + rfi_pos_mod)%26
    chri = entry[rf_in]
    mirrored = reflectors[rflector][chri]
    
    print(f"{forward_run} -> {chri} (into reflector) -> {mirrored} (reflected out)")
    
    ## second run back left to right thru scrambler
    back_run = once_thru_scramble(mirrored, direction='back', first_rotor=left_rotor, pos1=lpos, 
                                  second_rotor=middle_rotor, pos2=mpos, third_rotor=right_rotor, pos3=rpos)
    
    bk_out = entry.index(back_run)
    bko_pos_mod = (26 + 0 - rpos)%26  ## as above, '0' just reflects that the entry interface doesn't move
    bk_final = (bk_out + bko_pos_mod)%26
    final = entry[bk_final]
    print('RR back out:  ', back_run, '-->', final)
    
    return final

# full_scramble('i')

In [9]:
full_scramble('q', lpos=2, mpos=22, rpos=10)

Q -> (RR out) E -> (MR in) Q -> (MR out) Q -> (LR in) W -> (LR out) U
U -> S (into reflector) -> F (reflected out)
F -> (LR out) D -> (MR in) X -> (MR out) I -> (RR in) W -> (RR out) N
RR back out:   N --> D


'D'

In [10]:
word = 'oh what a beautiful morning oh what a beautiful day i have a wonderful feeling everthings going my way'

In [11]:
def step_rotor(p_in):
    """"""
    if p_in == 25:
        p_in = 0
    else:
        p_in += 1
    return p_in

In [12]:
def only_ascii(instring):
    """strips out anything that's not an ascii character (i.e a-z alphabet character), capitalises"""
    newstring = ''
    for character in instring:
        if character in ascii_letters:
            newstring += character.upper()
    return newstring

In [13]:
only_ascii(word)

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'

In [14]:
sht = 'pat'
sht = sht.upper()

lt, mt, rt = (ascii_uppercase.index(s) for s in sht)
print(lt, mt, rt)

15 0 19


In [15]:
rt

19

In [16]:
def enigmatise(tocode, startset='AAA', lstartpos=0, mstartpos=0, rstartpos=0):
    encoded = ''

    tocode = only_ascii(tocode)
    
    lftpos, midpos, rgtpos = (ascii_uppercase.index(s) for s in startset.upper())
    
    
    # start out initial positions as starting positions
#     lftpos = lstartpos
#     midpos = mstartpos
#     rgtpos = rstartpos

    ## TO DO need to enable notches to be defined by ring setting as input to function
    mnotch = 4   ## point if right rotor reaches will trigger middle rotor to step
    lnotch = 5  ## point if middle rotor reaches will trigger left rotor to step

    for c in tocode:
        print('in:    ', c)

        ## simple stepping version - notches step M & L straight away
#         rgtpos = step_rotor(rgtpos)
#         if rgtpos == mnotch:
#             midpos = step_rotor(midpos)
#             print('--- middle rotor step ---')
#             if midpos == lnotch:
#                 lftpos = step_rotor(lftpos)
#                 print('---  left rotor step  ---')
                
        rgtpos = step_rotor(rgtpos)
        
        if rgtpos == mnotch:
            
            midpos = step_rotor(midpos)
            print('--- middle rotor step ---')
            if midpos == lnotch:
                lftpos = step_rotor(lftpos)
                print('---  left rotor step  ---')

        print('rp = ', rgtpos, 'mp = ', midpos, 'lp = ', lftpos)
        out = full_scramble(c, lpos=lftpos, mpos=midpos, rpos=rgtpos)
        print('out:   ', out,'\n')
        encoded += out

    return encoded

In [17]:
opcode = 'ind'
## operator set starting position, traditionally different for each new message

In [18]:
igged = enigmatise(word, startset=opcode)
igged

in:     O
--- middle rotor step ---
rp =  4 mp =  14 lp =  8
O -> (RR out) S -> (MR in) C -> (MR out) D -> (LR in) X -> (LR out) S
S -> K (into reflector) -> N (reflected out)
N -> (LR out) L -> (MR in) R -> (MR out) G -> (RR in) W -> (RR out) N
RR back out:   N --> J
out:    J 

in:     H
rp =  5 mp =  14 lp =  8
H -> (RR out) O -> (MR in) X -> (MR out) V -> (LR in) P -> (LR out) E
E -> W (into reflector) -> V (reflected out)
V -> (LR out) B -> (MR in) H -> (MR out) L -> (RR in) C -> (RR out) Y
RR back out:   Y --> T
out:    T 

in:     W
rp =  6 mp =  14 lp =  8
W -> (RR out) M -> (MR in) U -> (MR out) P -> (LR in) J -> (LR out) T
T -> L (into reflector) -> G (reflected out)
G -> (LR out) Z -> (MR in) F -> (MR out) W -> (RR in) O -> (RR out) M
RR back out:   M --> G
out:    G 

in:     H
rp =  7 mp =  14 lp =  8
H -> (RR out) Y -> (MR in) F -> (MR out) I -> (LR in) C -> (LR out) F
F -> X (into reflector) -> J (reflected out)
J -> (LR out) I -> (MR in) O -> (MR out) Y -> (RR in) R -> 

rp =  0 mp =  16 lp =  8
G -> (RR out) D -> (MR in) T -> (MR out) N -> (LR in) F -> (LR out) L
L -> D (into reflector) -> H (reflected out)
H -> (LR out) H -> (MR in) P -> (MR out) U -> (RR in) E -> (RR out) A
RR back out:   A --> A
out:    A 

in:     O
rp =  1 mp =  16 lp =  8
O -> (RR out) H -> (MR in) W -> (MR out) F -> (LR in) X -> (LR out) S
S -> K (into reflector) -> N (reflected out)
N -> (LR out) L -> (MR in) T -> (MR out) N -> (RR in) Y -> (RR out) O
RR back out:   O --> N
out:    N 

in:     I
rp =  2 mp =  16 lp =  8
I -> (RR out) N -> (MR in) B -> (MR out) J -> (LR in) B -> (LR out) D
D -> V (into reflector) -> W (reflected out)
W -> (LR out) P -> (MR in) X -> (MR out) I -> (RR in) U -> (RR out) R
RR back out:   R --> P
out:    P 

in:     N
rp =  3 mp =  16 lp =  8
N -> (RR out) X -> (MR in) K -> (MR out) L -> (LR in) D -> (LR out) H
H -> Z (into reflector) -> T (reflected out)
T -> (LR out) A -> (MR in) I -> (MR out) F -> (RR in) S -> (RR out) S
RR back out:   S --> P
ou

'JTGQWEGDAOOBREYWDWZFTUKYCBIKDYLKWOVRTLBQCPJDZLTNYLKBUSZXOXUFBTGYSZZCMQLVQUANPPVJNRFM'

## Have now set up function 'enigmatise' to encode a message in enigma format
### Next step is to work on decoding it!

In [19]:
igged

'JTGQWEGDAOOBREYWDWZFTUKYCBIKDYLKWOVRTLBQCPJDZLTNYLKBUSZXOXUFBTGYSZZCMQLVQUANPPVJNRFM'

In [20]:
reword = enigmatise(igged, startset=opcode)
reword

in:     J
--- middle rotor step ---
rp =  4 mp =  14 lp =  8
J -> (RR out) W -> (MR in) G -> (MR out) R -> (LR in) L -> (LR out) V
V -> N (into reflector) -> K (reflected out)
K -> (LR out) X -> (MR in) D -> (MR out) C -> (RR in) S -> (RR out) S
RR back out:   S --> O
out:    O 

in:     T
rp =  5 mp =  14 lp =  8
T -> (RR out) C -> (MR in) L -> (MR out) H -> (LR in) B -> (LR out) D
D -> V (into reflector) -> W (reflected out)
W -> (LR out) P -> (MR in) V -> (MR out) X -> (RR in) O -> (RR out) M
RR back out:   M --> H
out:    H 

in:     G
rp =  6 mp =  14 lp =  8
G -> (RR out) O -> (MR in) W -> (MR out) F -> (LR in) Z -> (LR out) O
O -> G (into reflector) -> L (reflected out)
L -> (LR out) J -> (MR in) P -> (MR out) U -> (RR in) M -> (RR out) C
RR back out:   C --> W
out:    W 

in:     Q
rp =  7 mp =  14 lp =  8
Q -> (RR out) R -> (MR in) Y -> (MR out) O -> (LR in) I -> (LR out) R
R -> J (into reflector) -> X (reflected out)
X -> (LR out) C -> (MR in) I -> (MR out) F -> (RR in) Y -> 

RR back out:   W --> A
out:    A 

in:     L
rp =  23 mp =  15 lp =  8
L -> (RR out) V -> (MR in) N -> (MR out) T -> (LR in) M -> (LR out) Z
Z -> R (into reflector) -> B (reflected out)
B -> (LR out) E -> (MR in) L -> (MR out) K -> (RR in) S -> (RR out) S
RR back out:   S --> V
out:    V 

in:     T
rp =  24 mp =  15 lp =  8
T -> (RR out) U -> (MR in) L -> (MR out) H -> (LR in) A -> (LR out) B
B -> T (into reflector) -> Z (reflected out)
Z -> (LR out) D -> (MR in) K -> (MR out) D -> (RR in) M -> (RR out) C
RR back out:   C --> E
out:    E 

in:     N
rp =  25 mp =  15 lp =  8
N -> (RR out) O -> (MR in) E -> (MR out) S -> (LR in) L -> (LR out) V
V -> N (into reflector) -> K (reflected out)
K -> (LR out) X -> (MR in) E -> (MR out) Z -> (RR in) J -> (RR out) Z
RR back out:   Z --> A
out:    A 

in:     Y
rp =  0 mp =  15 lp =  8
Y -> (RR out) C -> (MR in) R -> (MR out) G -> (LR in) Z -> (LR out) O
O -> G (into reflector) -> L (reflected out)
L -> (LR out) J -> (MR in) Q -> (MR out) Q -> (

'OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY'

In [21]:
print(igged)
print(reword)

JTGQWEGDAOOBREYWDWZFTUKYCBIKDYLKWOVRTLBQCPJDZLTNYLKBUSZXOXUFBTGYSZZCMQLVQUANPPVJNRFM
OHWHATABEAUTIFULMORNINGOHWHATABEAUTIFULDAYIHAVEAWONDERFULFEELINGEVERTHINGSGOINGMYWAY


In [22]:
crib = 'OHWHATABEAUTIFULDAY'
lncrib = len(crib)

In [23]:
reword[23:23+lncrib]

'OHWHATABEAUTIFULDAY'

In [24]:
igged[:len(igged)-lncrib]

'JTGQWEGDAOOBREYWDWZFTUKYCBIKDYLKWOVRTLBQCPJDZLTNYLKBUSZXOXUFBTGYS'

#### Have chosen a crib (how to do this automatically will be a bigger problem later on)
#### first step is to find positions in the encoded text ('igged') that could feasibly contain the crib words, as defined by not encoding a letter as itself

In [25]:

status_of_crib_indexes = {k:True for k in range(len(igged))}
# print(status_of_crib_indexes)
for num, letter in enumerate(igged[:len(igged)-lncrib]):
    trange = range(num,num+lncrib)
    tcrib = igged[num:num+lncrib]
    for c, t in zip(crib, tcrib):
        if c == t:
            status_of_crib_indexes[num] = False
    
        
print(status_of_crib_indexes)
maybes = [k for k,v in status_of_crib_indexes.items() if v == True]
print(maybes)

{0: False, 1: True, 2: False, 3: True, 4: False, 5: False, 6: False, 7: False, 8: True, 9: False, 10: False, 11: False, 12: False, 13: False, 14: False, 15: False, 16: True, 17: True, 18: False, 19: True, 20: True, 21: True, 22: False, 23: True, 24: True, 25: False, 26: True, 27: False, 28: True, 29: True, 30: False, 31: False, 32: True, 33: False, 34: False, 35: False, 36: True, 37: True, 38: False, 39: True, 40: True, 41: False, 42: False, 43: True, 44: False, 45: False, 46: False, 47: True, 48: False, 49: True, 50: False, 51: True, 52: True, 53: False, 54: True, 55: False, 56: False, 57: False, 58: True, 59: False, 60: True, 61: True, 62: True, 63: False, 64: True, 65: True, 66: True, 67: True, 68: True, 69: True, 70: True, 71: True, 72: True, 73: True, 74: True, 75: True, 76: True, 77: True, 78: True, 79: True, 80: True, 81: True, 82: True, 83: True}
[1, 3, 8, 16, 17, 19, 20, 21, 23, 24, 26, 28, 29, 32, 36, 37, 39, 40, 43, 47, 49, 51, 52, 54, 58, 60, 61, 62, 64, 65, 66, 67, 68, 69,

#### Looks like there are many potential positions. This will be a problem later in the piece when trying to do more unknown messages. Will test out a method for a crib that I know is in the correct position 

In [26]:
mcrib = igged[23:23+lncrib]
print(crib)
print(mcrib)

OHWHATABEAUTIFULDAY
YCBIKDYLKWOVRTLBQCP


In [27]:
pairs = {i:{c,m} for i,c,m in zip(range(23+lncrib), crib, mcrib)}
pairs

{0: {'O', 'Y'},
 1: {'C', 'H'},
 2: {'B', 'W'},
 3: {'H', 'I'},
 4: {'A', 'K'},
 5: {'D', 'T'},
 6: {'A', 'Y'},
 7: {'B', 'L'},
 8: {'E', 'K'},
 9: {'A', 'W'},
 10: {'O', 'U'},
 11: {'T', 'V'},
 12: {'I', 'R'},
 13: {'F', 'T'},
 14: {'L', 'U'},
 15: {'B', 'L'},
 16: {'D', 'Q'},
 17: {'A', 'C'},
 18: {'P', 'Y'}}

In [28]:
links = {character:len([pair for pair in pairs.values() if character in pair]) for character in entry}   
print(links)
hilinks = {k:v for k,v in links.items() if v > 1}
hilinks

{'A': 4, 'B': 3, 'C': 2, 'D': 2, 'E': 1, 'F': 1, 'G': 0, 'H': 2, 'I': 2, 'J': 0, 'K': 2, 'L': 3, 'M': 0, 'N': 0, 'O': 2, 'P': 1, 'Q': 1, 'R': 1, 'S': 0, 'T': 3, 'U': 2, 'V': 1, 'W': 2, 'X': 0, 'Y': 3, 'Z': 0}


{'A': 4,
 'B': 3,
 'C': 2,
 'D': 2,
 'H': 2,
 'I': 2,
 'K': 2,
 'L': 3,
 'O': 2,
 'T': 3,
 'U': 2,
 'W': 2,
 'Y': 3}

#### 'links' is just record for each letter of the alphabet, how many times it appears paired to a cypher or natural letter in either of the encoded or raw messages
#### 'hilinks' is just those letters that have more than 2 pairs (links)
#### 'hipairs' is now preparing, for each of the 'hilinks' letters of interest, a dictionary record of which other letters they are paired to, and in what position

In [45]:
mostlinks = sorted(hilinks.values(),reverse=True)[0]
mostlinks

4

In [46]:
best_characters = [k for k,v in hilinks.items() if v == mostlinks]
best_characters

['A']

In [29]:
hipairs = {}
for character in hilinks.keys():
    hset = {k:list(pair) for k,pair in pairs.items() if character in pair}
    newresult = {}
    for k,v in hset.items():
        v.remove(character)
        if v[0] in hilinks.keys():
            newresult[k] = v[0]
#     print('hset:      ',hset)
#     print('newresult: ',newresult)
    if len(newresult) > 0:
        hipairs[character] = newresult
#     hipairs[character] = {k:pair.remove(character) for k,pair in pairs.items() if character in pair}
## maybe try a comprehension again later, involves a few steps...

    
hipairs

{'A': {4: 'K', 6: 'Y', 9: 'W', 17: 'C'},
 'B': {2: 'W', 7: 'L', 15: 'L'},
 'C': {1: 'H', 17: 'A'},
 'D': {5: 'T'},
 'H': {1: 'C', 3: 'I'},
 'I': {3: 'H'},
 'K': {4: 'A'},
 'L': {7: 'B', 14: 'U', 15: 'B'},
 'O': {0: 'Y', 10: 'U'},
 'T': {5: 'D'},
 'U': {10: 'O', 14: 'L'},
 'W': {2: 'B', 9: 'A'},
 'Y': {0: 'O', 6: 'A'}}

#### for now just doing this entire process for one letter, 'a', set as variable 'x'. Will need to put this whole process into a function later so it can iterate through each of the letters - I think just the letters in hilinks, as opposed to every letter in the alphabet
Though thinking further - maybe it is alright to just start with the letter with the most connections? The chain space for a given cypher/crib set of characters will be the same, the starting letter is just to give the function somewhere to start?

In [30]:
def make_connections(x, dx, loops={}, deadends={}, itr = 1):
    """for sorting through a hipairs dictionary of letters of interest and their corresponding paired letters. 
    Used with a WHILE loop, can recursively search through 'chains' or paths that a letter sequence can take 
    by following pairs from one letter to the next. Looks for 'loops', where a chain path can return to its original
    starting letter. Records other chains as deadends
    """
    working_dict = deepcopy(dx)
    # print(nd)
    for iD,chain in dx.items():
        current_end = chain[-1]
        cx = hipairs[current_end]
#         print(cx)
        for jid,conxn in enumerate(cx.values()):
            key = round(iD+jid/10**itr,5)

            if conxn != current_end:
                working_dict[key] = dx[iD] + conxn
    #     print(cx)
    print('all connections')
    pprint(working_dict)

    dx={}
    for kid,chain in working_dict.items():
        if chain[-1] == chain[-3]:# and len(v):
            chain = chain[:-1]
            deadends[kid] = chain
        elif chain[-1] == x and len(chain) > 3:   ## ie we're legit back to the start after a loop
            loops[kid] = chain
        else:
            dx[kid] = chain
    print('useful connections: ')
    pprint(dx)
    print('loops:')
    pprint(loops)
    return dx, loops, deadends

In [31]:
def find_loops(starting_character='A'):
    working_dict = {i+0.0:starting_character for i in range(len(hipairs[starting_character]))}
    for i,v in zip(range(len(hipairs[starting_character])), hipairs[starting_character].values()):
        working_dict[i] += v
#     pprint(ax)

    run = 1
    found_loops = {}
    dead_ends = {}
    while len(working_dict) > 0:
        print(f"run {run}")
        working_dict, found_loops, dead_ends = make_connections(starting_character, working_dict,found_loops,dead_ends,run)
        run +=1
    
    return found_loops, dead_ends

In [47]:
aloops, d_ends = find_loops(best_characters[0])

run 1
all connections
{0.0: 'AKA',
 1.0: 'AYO',
 1.1: 'AYA',
 2.0: 'AWB',
 2.1: 'AWA',
 3.0: 'ACH',
 3.1: 'ACA'}
useful connections: 
{1.0: 'AYO', 2.0: 'AWB', 3.0: 'ACH'}
loops:
{}
run 2
all connections
{1.0: 'AYOY',
 1.01: 'AYOU',
 2.0: 'AWBW',
 2.01: 'AWBL',
 2.02: 'AWBL',
 3.0: 'ACHC',
 3.01: 'ACHI'}
useful connections: 
{1.01: 'AYOU', 2.01: 'AWBL', 2.02: 'AWBL', 3.01: 'ACHI'}
loops:
{}
run 3
all connections
{1.01: 'AYOUO',
 1.011: 'AYOUL',
 2.01: 'AWBLB',
 2.011: 'AWBLU',
 2.012: 'AWBLB',
 2.02: 'AWBLB',
 2.021: 'AWBLU',
 2.022: 'AWBLB',
 3.01: 'ACHIH'}
useful connections: 
{1.011: 'AYOUL', 2.011: 'AWBLU', 2.021: 'AWBLU'}
loops:
{}
run 4
all connections
{1.011: 'AYOULB',
 1.0111: 'AYOULU',
 1.0112: 'AYOULB',
 2.011: 'AWBLUO',
 2.0111: 'AWBLUL',
 2.021: 'AWBLUO',
 2.0211: 'AWBLUL'}
useful connections: 
{1.011: 'AYOULB', 1.0112: 'AYOULB', 2.011: 'AWBLUO', 2.021: 'AWBLUO'}
loops:
{}
run 5
all connections
{1.011: 'AYOULBW',
 1.01101: 'AYOULBL',
 1.01102: 'AYOULBL',
 1.0112: 'AYOULBW',


In [48]:
aloops

{1.011: 'AYOULBWA', 2.011: 'AWBLUOYA', 2.021: 'AWBLUOYA', 1.0112: 'AYOULBWA'}

In [49]:
d_ends

{0.0: 'AK',
 1.1: 'AY',
 2.1: 'AW',
 3.1: 'AC',
 1.0: 'AYO',
 2.0: 'AWB',
 3.0: 'ACH',
 1.01: 'AYOU',
 2.01: 'AWBL',
 2.02: 'AWBL',
 3.01: 'ACHI',
 2.012: 'AWBL',
 2.022: 'AWBL',
 1.0111: 'AYOUL',
 2.0111: 'AWBLU',
 2.0211: 'AWBLU',
 1.01101: 'AYOULB',
 1.01102: 'AYOULB',
 2.01101: 'AWBLUO',
 2.02101: 'AWBLUO',
 1.01121: 'AYOULB',
 1.01122: 'AYOULB'}

In [50]:
def rationalise_loops(indict):
    invals = list(set(indict.values()))
#     invals = list(set(invals))
    for i,loop in enumerate(invals):
        test = deepcopy(invals)
#         print(test)
        test[i] = loop[::-1]
        test = list(set(test))
#         print(test)
        if len(test) != len(invals):
            invals[i] = loop[::-1]
#         print(invals)
    invals = list(set(invals))

    return invals

In [51]:
rdends = rationalise_loops(d_ends)
print(rdends)

['AYO', 'AWBL', 'AWB', 'AC', 'ACHI', 'ACH', 'AW', 'AY', 'AK', 'AYOUL', 'AYOU', 'AYOULB', 'AWBLU', 'AWBLUO']


In [52]:
def unsub_list(inlist):
    inlist.sort()
    unique = []
    for i in range(len(inlist)-1):
        chain = inlist[i]
        next_chain = inlist[i+1]
#         print(chain, next_chain, chain in next_chain)
        if chain not in next_chain:
            unique.append(chain)

    unique.append(inlist[-1])

    return unique

In [53]:
uqends = unsub_list(rdends)
uqends

['ACHI', 'AK', 'AWBLUO', 'AYOULB']

In [54]:
aloops

{1.011: 'AYOULBWA', 2.011: 'AWBLUOYA', 2.021: 'AWBLUOYA', 1.0112: 'AYOULBWA'}

In [55]:
rloops = rationalise_loops(aloops)
rloops

['AWBLUOYA']

['AYOULBWA']

In [56]:
lcheck = set(aloops.values())
print(lcheck)
fuqends = deepcopy(uqends)
for uchain in uqends:
#     print(uchain)
    for l in lcheck:
        print (uchain, l, uchain in l)
        if uchain in l:
            fuqends.remove(uchain)
            
fuqends

{'AYOULBWA', 'AWBLUOYA'}
ACHI AYOULBWA False
ACHI AWBLUOYA False
AK AYOULBWA False
AK AWBLUOYA False
AWBLUO AYOULBWA False
AWBLUO AWBLUOYA True
AYOULB AYOULBWA True
AYOULB AWBLUOYA False


['ACHI', 'AK']

['ACHI', 'AK']

In [59]:
hipairs

{'A': {4: 'K', 6: 'Y', 9: 'W', 17: 'C'},
 'B': {2: 'W', 7: 'L', 15: 'L'},
 'C': {1: 'H', 17: 'A'},
 'D': {5: 'T'},
 'H': {1: 'C', 3: 'I'},
 'I': {3: 'H'},
 'K': {4: 'A'},
 'L': {7: 'B', 14: 'U', 15: 'B'},
 'O': {0: 'Y', 10: 'U'},
 'T': {5: 'D'},
 'U': {10: 'O', 14: 'L'},
 'W': {2: 'B', 9: 'A'},
 'Y': {0: 'O', 6: 'A'}}

In [57]:
rloops

['AWBLUOYA']

In [75]:
mainloop = rloops[0]
menu= {}
for i,char in enumerate(mainloop[:-1]):
    next_char = mainloop[i+1]
    wdict = hipairs[char]
#     print(wdict)
    position = [k for k,v in wdict.items() if v == next_char][0]
    ### note that I'm just picking the first one where there are double (or more) linkages
    ### not sure if this matters for now or if its better to somehow include both linkages in the menu
    ### revisit later depending on bombe methodology
#     print(position)
    print(i,char,next_char,position)
    menu[i] = (char, next_char, position)
menu

0 A W 9
1 W B 2
2 B L 7
3 L U 14
4 U O 10
5 O Y 0
6 Y A 6


{0: ('A', 'W', 9),
 1: ('W', 'B', 2),
 2: ('B', 'L', 7),
 3: ('L', 'U', 14),
 4: ('U', 'O', 10),
 5: ('O', 'Y', 0),
 6: ('Y', 'A', 6)}